# Using submodels in PyBaMM
In this notebook we show how to modify existing models by swapping out submodels, and how to build your own model from scratch using existing submodels. To see all of the models and submodels available in PyBaMM, please take a look at the documentation [here](https://pybamm.readthedocs.io/en/latest/source/models/index.html).

## Changing a submodel in an exisiting battery model
PyBaMM is designed to be a flexible modelling package that allows users to easily compare different models and numerical techniques within a common framework. Battery models within PyBaMM are built up using a number of submodels that describe different physics included within the model, such as mass conservation in the electrolyte or charge conservation in the solid. For ease of use, a number of popular battery models are pre-configured in PyBaMM. As an example, we look at the Single Particle Model (for more information see [here](./models/SPM.ipynb)). 

First we import pybamm

In [1]:
%pip install pybamm -q    # install PyBaMM if it is not installed
import pybamm

Note: you may need to restart the kernel to use updated packages.


Then we load the SPM

In [2]:
model = pybamm.lithium_ion.SPM()

We can look at the submodels that make up the SPM by accessing `model.submodels`, which is a dictionary of the submodel's name (i.e. the physics it represents) and the submodel that is selected

In [3]:
for name, submodel in model.submodels.items():
    print(name, submodel)

external circuit <pybamm.models.submodels.external_circuit.explicit_control_external_circuit.ExplicitCurrentControl object at 0x7f93eb102610>
porosity <pybamm.models.submodels.porosity.constant_porosity.Constant object at 0x7f93da6ecd90>
negative interface utilisation <pybamm.models.submodels.interface.interface_utilisation.full_utilisation.Full object at 0x7f93da6ecf40>
positive interface utilisation <pybamm.models.submodels.interface.interface_utilisation.full_utilisation.Full object at 0x7f93da6f90d0>
negative particle mechanics <pybamm.models.submodels.particle_mechanics.no_mechanics.NoMechanics object at 0x7f93da6f9220>
positive particle mechanics <pybamm.models.submodels.particle_mechanics.no_mechanics.NoMechanics object at 0x7f93da6f93a0>
negative primary active material <pybamm.models.submodels.active_material.constant_active_material.Constant object at 0x7f93da6f94f0>
positive primary active material <pybamm.models.submodels.active_material.constant_active_material.Constant ob

When you load a model in PyBaMM it builds by default. Building the model sets all of the model variables and sets up any variables which are coupled between different submodels: this is the process which couples the submodels together and allows one submodel to access variables from another. If you would like to swap out a submodel in an existing battery model you need to load it without building it by passing the keyword `build=False`

In [4]:
model = pybamm.lithium_ion.SPM(build=False)

This collects all of the submodels which make up the SPM, but doesn't build the model. Now you are free to swap out one submodel for another. For instance, you may want to assume that diffusion within the negative particles is infinitely fast, so that the PDE describing diffusion is replaced with an ODE for the uniform particle concentration. To change a submodel you simply update the dictionary entry, in this case to the `XAveragedPolynomialProfile` submodel

In [5]:
model.submodels["negative primary particle"] = pybamm.particle.XAveragedPolynomialProfile(
    model.param, "Negative", options={**model.options, "particle": "uniform profile"}
)

where we pass in the model parameters, the electrode (negative or positive) the submodel corresponds to, and the name of the polynomial we want to use. In the example we assume uniform concentration within the particle, corresponding to a zero-order polynomial.

Now if we look at the submodels again we see that the model for the negative particle has been changed

In [6]:
for name, submodel in model.submodels.items():
    print(name, submodel)

external circuit <pybamm.models.submodels.external_circuit.explicit_control_external_circuit.ExplicitCurrentControl object at 0x7f93da764ac0>
porosity <pybamm.models.submodels.porosity.constant_porosity.Constant object at 0x7f93c88a9400>
negative interface utilisation <pybamm.models.submodels.interface.interface_utilisation.full_utilisation.Full object at 0x7f93c88a0ca0>
positive interface utilisation <pybamm.models.submodels.interface.interface_utilisation.full_utilisation.Full object at 0x7f93c88a0850>
negative particle mechanics <pybamm.models.submodels.particle_mechanics.no_mechanics.NoMechanics object at 0x7f93c88a0400>
positive particle mechanics <pybamm.models.submodels.particle_mechanics.no_mechanics.NoMechanics object at 0x7f93c88a04f0>
negative primary active material <pybamm.models.submodels.active_material.constant_active_material.Constant object at 0x7f93c8894ac0>
positive primary active material <pybamm.models.submodels.active_material.constant_active_material.Constant ob

Building the model also sets up the equations, boundary and initial conditions for the model. For example, if we look at `model.rhs` before building we see that it is empty 

In [7]:
model.rhs

{}

If we try to use this empty model, PyBaMM will give an error. So, before proceeding we must build the model

In [8]:
model.build_model()

Now if we look at `model.rhs` we see that it contains an entry relating to the concentration in each particle, as expected for the SPM

In [9]:
model.rhs

{Variable(-0x3424fcee14f4853a, Discharge capacity [A.h], children=[], domains={}): Division(-0x42460e77ad8fefc6, /, children=['Current function [A] * 96485.33212 * Maximum concentration in negative electrode [mol.m-3] * (Negative electrode thickness [m] + Separator thickness [m] + Positive electrode thickness [m]) / absolute(Typical current [A] / (Number of electrodes connected in parallel to make a cell * Electrode width [m] * Electrode height [m]))', '3600.0'], domains={}),
 Variable(-0x51cb6922fc1868b9, Average negative particle concentration, children=[], domains={'primary': ['current collector']}): MatrixMultiplication(-0x70e9ee195fdfc364, @, children=['mass(Average negative particle concentration)', '-3.0 * (Current function [A] / Typical current [A]) * sign(Typical current [A]) / ((3.0 * x-average(Negative electrode active material volume fraction) / x-average(Negative particle radius [m]) / (3.0 * yz-average(x-average(Negative electrode active material volume fraction)) / yz-av

Now the model can be used in a simulation and solved in the usual way, and we still have access to model defaults such as the default geometry and default spatial methods which are used in the simulation

In [10]:
simulation = pybamm.Simulation(model)
simulation.solve([0, 3600])
simulation.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

## Building a custom model from submodels
Instead of editing a pre-existing model, you may wish to build your own model from scratch by combining existing submodels of you choice. In this section, we build a Single Particle Model in which the diffusion is assumed infinitely fast in both particles. 

To begin, we load a base lithium-ion model. This sets up the basic model structure behind the scenes, and also sets the default parameters to be those corresponding to a lithium-ion battery. Note that the base model does not select any default submodels, so there is no need to pass `build=False`.

In [11]:
model = pybamm.lithium_ion.BaseModel()

Submodels can be added to the `model.submodels` dictionary in the same way that we changed the submodels earlier. 

We use the simplest model for the external circuit, which is the explicit "current control" submodel

In [12]:
model.submodels["external circuit"] = pybamm.external_circuit.ExplicitCurrentControl(model.param, model.options)

We want to build a 1D model, so select the `Uniform` current collector model (if the current collectors are behaving uniformly, then a 1D model is appropriate). We also want the model to be isothermal, so select the thermal model accordingly. Further, we assume that the porosity and active material are constant in space and time.

In [13]:
model.submodels["current collector"] = pybamm.current_collector.Uniform(model.param)
model.submodels["thermal"] = pybamm.thermal.isothermal.Isothermal(model.param)
model.submodels["porosity"] = pybamm.porosity.Constant(model.param, model.options)
model.submodels["negative active material"] = pybamm.active_material.Constant(
    model.param, "Negative", model.options
)
model.submodels["positive active material"] = pybamm.active_material.Constant(
    model.param, "Positive", model.options
)

We assume that the current density varies linearly in the electrodes. This corresponds the the leading-order terms in Ohm's law in the limit in which the SPM is derived in [[3]](#References)

In [14]:
model.submodels["negative electrode potentials"] = pybamm.electrode.ohm.LeadingOrder(
    model.param, "Negative"
)
model.submodels["positive electrode potentials"] = pybamm.electrode.ohm.LeadingOrder(
    model.param, "Positive"
)

We assume uniform concentration in both the negative and positive particles 

In [15]:
options = {**model.options, "particle": "uniform profile"}
model.submodels["negative primary particle"] = pybamm.particle.XAveragedPolynomialProfile(model.param, "Negative", options)
model.submodels["positive primary particle"] = pybamm.particle.XAveragedPolynomialProfile(model.param, "Positive", options)

In the Single Particle Model, the overpotential can be obtained by inverting the Butler-Volmer relation, so we choose the `InverseButlerVolmer` submodel for the interface, with the "main" lithium-ion reaction (and default lithium ion options). Because of how the current is implemented, we also need to separately specify the `CurrentForInverseButlerVolmer` submodel. We also need to specify the submodel for open-circuit potential.

In [16]:
model.submodels[
    "negative open circuit potential"
] = pybamm.open_circuit_potential.SingleOpenCircuitPotential(
    model.param, "Negative", "lithium-ion main", options=model.options
)
model.submodels[
    "positive open circuit potential"
] = pybamm.open_circuit_potential.SingleOpenCircuitPotential(
    model.param, "Positive", "lithium-ion main", options=model.options
)
model.submodels[
    "negative interface"
] = pybamm.kinetics.InverseButlerVolmer(
    model.param, "Negative", "lithium-ion main", options=model.options
)
model.submodels[
    "positive interface"
] = pybamm.kinetics.InverseButlerVolmer(
    model.param, "Positive", "lithium-ion main", options=model.options
)
model.submodels[
    "negative interface current"
] = pybamm.kinetics.CurrentForInverseButlerVolmer(
    model.param, "Negative", "lithium-ion main"
)
model.submodels[
    "positive interface current"
] = pybamm.kinetics.CurrentForInverseButlerVolmer(
    model.param, "Positive", "lithium-ion main"
)
model.submodels["negative interface utilisation"] = pybamm.interface_utilisation.Full(
    model.param, "Negative", model.options
)
model.submodels["positive interface utilisation"] = pybamm.interface_utilisation.Full(
    model.param, "Positive", model.options
)

We don't want any particle mechanics, SEI formation or lithium plating in this model

In [17]:
model.submodels[
    "Negative particle mechanics"
] = pybamm.particle_mechanics.NoMechanics(model.param, "Negative", model.options)
model.submodels[
    "Positive particle mechanics"
] = pybamm.particle_mechanics.NoMechanics(model.param, "Positive", model.options)
model.submodels["sei"] = pybamm.sei.NoSEI(model.param, model.options)
model.submodels["sei on cracks"] = pybamm.sei.NoSEI(model.param, model.options, cracks=True)
model.submodels["lithium plating"] = pybamm.lithium_plating.NoPlating(model.param)

Finally, for the electrolyte we assume that diffusion is infinitely fast so that the concentration is uniform, and also use the leading-order model for charge conservation, which leads to a linear variation in ionic current in the electrodes

In [18]:
model.submodels["electrolyte diffusion"] = pybamm.electrolyte_diffusion.ConstantConcentration(
    model.param
)
model.submodels["electrolyte conductivity"] = pybamm.electrolyte_conductivity.LeadingOrder(
    model.param
)

Now that we have set all of the submodels we can build the model

In [19]:
model.build_model()

We can then use the model in a simulation in the usual way

In [20]:
simulation = pybamm.Simulation(model)
simulation.solve([0, 3600])
simulation.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…

## References

The relevant papers for this notebook are:

In [21]:
pybamm.print_citations()

[1] Weilong Ai, Ludwig Kraft, Johannes Sturm, Andreas Jossen, and Billy Wu. Electrochemical thermal-mechanical modelling of stress inhomogeneity in lithium-ion pouch cells. Journal of The Electrochemical Society, 167(1):013512, 2019. doi:10.1149/2.0122001JES.
[2] Joel A. E. Andersson, Joris Gillis, Greg Horn, James B. Rawlings, and Moritz Diehl. CasADi – A software framework for nonlinear optimization and optimal control. Mathematical Programming Computation, 11(1):1–36, 2019. doi:10.1007/s12532-018-0139-4.
[3] Rutooj Deshpande, Mark Verbrugge, Yang-Tse Cheng, John Wang, and Ping Liu. Battery cycle life prediction with coupled chemical degradation and fatigue mechanics. Journal of the Electrochemical Society, 159(10):A1730, 2012. doi:10.1149/2.049210jes.
[4] Charles R. Harris, K. Jarrod Millman, Stéfan J. van der Walt, Ralf Gommers, Pauli Virtanen, David Cournapeau, Eric Wieser, Julian Taylor, Sebastian Berg, Nathaniel J. Smith, and others. Array programming with NumPy. Nature, 585(782